In [17]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import subprocess

In [18]:
num_outputs = 318

In [ ]:
# for i in range(num_outputs):
#     file_in = f"images/failfint_{i:04d}.ppm"
#     file_out = f"vids/ffint_{i:04d}.png"
#     im = Image.open(file_in)
#     im.save(file_out, "PNG")
#     file_in = f"images/failgamc_{i:04d}.ppm"
#     file_out = f"vids/fgamc_{i:04d}.png"
#     im = Image.open(file_in)
#     im.save(file_out, "PNG")
#     file_in = f"images/failu2p3_{i:04d}.ppm"
#     file_out = f"vids/fu2p3_{i:04d}.png"
#     im = Image.open(file_in)
#     im.save(file_out, "PNG")
#     file_in = f"images/failu2p2_{i:04d}.ppm"
#     file_out = f"vids/fu2p2_{i:04d}.png"
#     im = Image.open(file_in)
#     im.save(file_out, "PNG")
#     file_in = f"images/failu2p1_{i:04d}.ppm"
#     file_out = f"vids/fu2p1_{i:04d}.png"
#     im = Image.open(file_in)
#     im.save(file_out, "PNG")

In [20]:
for i in range(num_outputs):
    file_in = f"images/im_lgam_{i:04d}.ppm"
    file_out = f"vids/im_lgam_{i:04d}.png"
    im = Image.open(file_in)
    im.save(file_out, "PNG")
    file_in = f"images/im_lbsq_{i:04d}.ppm"
    file_out = f"vids/im_lbsq_{i:04d}.png"
    im = Image.open(file_in)
    im.save(file_out, "PNG")
    file_in = f"images/im_lu_{i:04d}.ppm"
    file_out = f"vids/im_lu_{i:04d}.png"
    im = Image.open(file_in)
    im.save(file_out, "PNG")
    file_in = f"images/im_lrho_{i:04d}.ppm"
    file_out = f"vids/im_lrho_{i:04d}.png"
    im = Image.open(file_in)
    im.save(file_out, "PNG")
    file_in = f"images/im_gam_{i:04d}.ppm"
    file_out = f"vids/im_gam_{i:04d}.png"
    im = Image.open(file_in)
    im.save(file_out, "PNG")
    file_in = f"images/im_bsq_{i:04d}.ppm"
    file_out = f"vids/im_bsq_{i:04d}.png"
    im = Image.open(file_in)
    im.save(file_out, "PNG")
    file_in = f"images/im_u_{i:04d}.ppm"
    file_out = f"vids/im_u_{i:04d}.png"
    im = Image.open(file_in)
    im.save(file_out, "PNG")
    file_in = f"images/im_rho_{i:04d}.ppm"
    file_out = f"vids/im_rho_{i:04d}.png"
    im = Image.open(file_in)
    im.save(file_out, "PNG")

In [21]:
fileList = ['im_lgam', 'im_lbsq', 'im_lu', 'im_lrho', 'im_gam', 'im_bsq', 'im_u', 'im_rho']

In [ ]:
for file in fileList:
    command = ['ffmpeg','-framerate','10','-i', f'vids/{file}_%04d.png','-c:v','libx264','-vf','pad=ceil(iw/2)*2:ceil(ih/2)*2',
            '-pix_fmt','yuv420p', f"vids/{file}.mp4", "-y"]
    result = subprocess.run(command)
subprocess.run('cd vids/; rm *.png')

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab